## Importação das Bibliotecas



In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Exportação
### 1 - Importando dados: Vinho - País e Ano

In [106]:
expo_vinho_ano = pd.read_excel('/content/exportacao_top_ano.xlsx')
expo_vinho_pais = pd.read_csv('/content/exportacao_vinho_agrupado_pais.csv')

# 1.1 Por ano

In [108]:
funcoes = {
    'Valor': 'sum',
    'Volume Litro': 'sum',
    'PIB US$': 'mean'
}

grouped_data = expo_vinho_ano.groupby('País').agg(funcoes)
grouped_data['PIB US$'] = grouped_data['PIB US$'].round(1)
grouped_data['Valor'] = grouped_data['Valor'].apply(lambda x: "${:,.2f}".format(x))
grouped_data['Volume Litro'] = grouped_data['Volume Litro'].apply(lambda x: "{:,.0f}".format(x).replace(",", "."))


In [109]:
maior_volume_ano = expo_vinho_ano

grouped_data = maior_volume_ano.sort_values(by='Valor', ascending=False)
grouped_data.drop_duplicates(subset=['País']).head(11).reset_index(drop=True)


,País,Ano,Volume Litro,Valor,PIB US$
0,Paraguai,2021,6522527,71923620,40000000000
1,Estados Unidos,2017,132688,15236990,19500000000000
2,Reino Unido,2014,305807,13737470,3000000000000
3,Haiti,2023,559645,8716610,19800000000
4,China,2016,134106,4996220,11200000000000


# 1.2 Por País

In [110]:
funcoes = {
    'Valor': 'sum',
    'Volume Litro': 'sum'
}

paises = {
    'Paraguai': 'Paraguai',
    'China': 'China',
    'Haiti': 'Haiti',
    'Estados Unidos': 'Estados Unidos',
    'Reino Unido': 'Reino Unido'
}

grouped_pais = expo_vinho_pais.groupby('País').agg(funcoes)
grouped_pais['Valor'] = grouped_pais['Valor'].apply(lambda x: "${:,.2f}".format(x))
grouped_pais['Volume Litro'] = grouped_pais['Volume Litro'].apply(lambda x: "{:,.0f}".format(x).replace(",", "."))
grouped_pais = grouped_pais.loc[list(paises.keys())]

grouped_pais

,Valor,Volume Litro
País,,
Paraguai,"$41,316,362.00",29.210.339
China,"$3,441,893.00",878.414
Haiti,"$3,189,236.00",2.344.028
Estados Unidos,"$6,836,115.00",2.294.640
Reino Unido,"$3,655,959.00",791.498


In [111]:
grouped_pais = expo_vinho_pais.sort_values(by='Valor', ascending=False)
merged_data = grouped_pais.merge(expo_vinho_ano, on='País', how='inner')
merged_data = merged_data.drop(columns=['Volume Litro_y', 'Valor_y'])
merged_data = merged_data.rename(columns={'Volume Litro_x': 'Volume Litro', 'Valor_x': 'Valor'})
merged_data["PIB US$"] = (merged_data["PIB US$"])
merged_data.drop_duplicates(subset=['País']).reset_index(drop=True)
merged_data.head()


,País,Volume Litro,Valor,Valor Litro,Temperatura,Ano,PIB US$
0,Paraguai,29210339,41316362.0,1.414443,23.92,2013,38700000000
1,Paraguai,29210339,41316362.0,1.414443,23.92,2014,40400000000
2,Paraguai,29210339,41316362.0,1.414443,23.92,2015,36200000000
3,Paraguai,29210339,41316362.0,1.414443,23.92,2016,36100000000
4,Paraguai,29210339,41316362.0,1.414443,23.92,2017,39000000000


In [141]:
agg_funcs = {
    'Volume Litro': 'sum',
    'Valor': 'sum',
    'Valor Litro': 'sum',
    'PIB US$': 'mean'
}

grouped_merged_data = merged_data.groupby('País').agg(agg_funcs)
grouped_merged_data = grouped_merged_data.sort_values(by='Valor', ascending=True)
grouped_merged_data['Valor'] = grouped_merged_data['Valor'].apply(lambda x: "${:,.2f}".format(x))
grouped_merged_data['Volume Litro'] = grouped_merged_data['Volume Litro'].apply(lambda x: "{:,.0f}".format(x).replace(",", "."))
grouped_merged_data['PIB US$'] = grouped_merged_data['PIB US$'].round(1)

grouped_merged_data

,Volume Litro,Valor,Valor Litro,PIB US$
País,,,,
Haiti,14.064.168,"$19,135,416.00",8.163476,1.776667e+10
China,9.662.554,"$37,860,823.00",43.101343,1.388182e+13
Reino Unido,8.706.478,"$40,215,549.00",50.809413,2.890909e+12
Estados Unidos,25.241.040,"$75,197,265.00",32.770833,2.086364e+13
Paraguai,321.313.729,"$454,479,982.00",15.558874,3.899091e+10


In [142]:
# prompt: gerar gráficos dinâmicos para grouped_merged_data.

import plotly.express as px



# Gráfico de barras interativo com Plotly Express
fig = px.bar(grouped_merged_data,
             x=grouped_merged_data.index,
             y='Valor',
             title='Valor Total de Exportação por País',
             labels={'x':'País', 'y':'Valor Total (USD)'},
             text='Valor') # Adiciona o valor em cima das barras

fig.show()

# Gráfico de barras para o Volume
fig_volume = px.bar(grouped_merged_data,
                   x=grouped_merged_data.index,
                   y='Volume Litro',
                   title='Volume Total Exportado por País',
                   labels={'x':'País', 'y':'Volume Total (Litros)'},
                   text='Volume Litro')

fig_volume.show()

# Gráfico de dispersão PIB vs Valor
fig_pib = px.scatter(grouped_merged_data,
                     x='PIB US$',
                     y='Valor',
                     title='PIB vs Valor de Exportação',
                     labels={'PIB US$':'PIB USD', 'Valor':'Valor de Exportação (USD)'},
                     hover_data=['Volume Litro'],
                     text=grouped_merged_data.index) # Adiciona o nome do país como texto
fig_pib.update_traces(textposition='top center') # Posiciona o texto
fig_pib.show()

### 2 - Análise Descritiva

### 3 - Análise de Vendas por País




### 4 - Análise PIB

### 5 - Análise de Regressão